# Classifying the language of the last name via RNN

The idea is to one hot encode characters and then create dense embeddings for them based upon some classification problem, such as predicting the next letter or predicting nationality of last name (a common example).

## Support code

In [ ]:
import pandas as pd
import numpy as np
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
#from torch.nn.functional import softmax
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
np.set_printoptions(precision=2, suppress=True, linewidth=3000, threshold=20000)
from typing import Sequence

dtype = torch.float
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def normal_transform(x, mean=0.0, std=0.01):
    "Convert x to have mean and std"
    return x*std + mean

def randn(n1, n2,
          device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
          dtype=torch.float,
          mean=0.0, std=0.01, requires_grad=False):
    x = torch.randn(n1, n2, device=device, dtype=dtype)
    x = normal_transform(x, mean=mean, std=std)
    x.requires_grad=requires_grad
    return x

In [ ]:
def plot_history(history, yrange=(0.0, 5.00), figsize=(3.5,3)):
    plt.figure(figsize=figsize)
    plt.ylabel("Sentiment log loss")
    plt.xlabel("Epochs")
    loss = history[:,0]
    valid_loss = history[:,1]
    plt.plot(loss, label='train_loss')
    plt.plot(valid_loss, label='val_loss')
    # plt.xlim(0, 200)
    plt.ylim(*yrange)
    plt.legend(loc='lower right')
    plt.show()

## Load

Let's download [training](https://raw.githubusercontent.com/hunkim/PyTorchZeroToAll/master/data/names_train.csv.gz) and [testing](https://raw.githubusercontent.com/hunkim/PyTorchZeroToAll/master/data/names_test.csv.gz) data for last names.   This data set is a bunch of last names and the nationality or language. 

In [ ]:
df_train = pd.read_csv("data/names_train.csv", header=None)
df_train.columns = ['name','language']
df_test = pd.read_csv("data/names_test.csv", header=None)
df_test.columns = ['name','language']

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_train.head(2)

## Clean

In [ ]:
badname = df_train['name']=='To The First Page'
df_train[badname]

In [ ]:
comma = df_train['name'].str.contains(',') # might as well keep
df_train[comma]

In [ ]:
df_train[df_train['name'].str.contains("'")][:3] # there are ok so keep quote

In [ ]:
badname = df_train['name']=='To The First Page'
df_train = df_train[~badname]

badname = df_test['name']=='To The First Page'
df_test = df_test[~badname]

In [ ]:
df_train['name'] = df_train['name'].str.lower()
df_test['name'] = df_test['name'].str.lower()

In [ ]:
def maxlen(strings:Sequence[str]) -> int:
    return max([len(l) for l in strings])

max_len = max(maxlen(df_train['name']), maxlen(df_test['name']))
max_len

## Split out validation set

In [ ]:
X, y = df_train[['name']], df_train['language']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20)
X_test, y_test = df_test[['name']], df_test['language']

## Get vocab

In [ ]:
def vocab(strings):
    letters = [list(l) for l in strings]
    V = set([c for cl in letters for c in cl])
    V = sorted(list(V))
    ctoi = {c:i for i, c in enumerate(V)}
    return V, ctoi

In [ ]:
V, ctoi = vocab(X['name'])
ctoi

## Encode target language (class)

Get categories from training only, not valid/test sets. Then apply cats to those set y's.

In [ ]:
y_train = y_train.astype('category').cat.as_ordered()
y_cats = y_train.cat.categories
y_cats

In [ ]:
y_train = y_train.cat.codes
y_train.values[:10]

In [ ]:
y_valid = pd.Categorical(y_valid, categories=y_cats, ordered=True).codes
y_test = pd.Categorical(y_test, categories=y_cats, ordered=True).codes

In [ ]:
y_valid[:5], y_test[:5]

## One-hot encode each letter of each name

Each name becomes a matrix of size vocab_size x max_len. Each column represents a char and we pad with zeros out to max_len number of columns since tensors have to be same length in same dimension. 

This approach is wasteful in that it expands each word to len of longest but avoids having to pad explicitly, simplifying the training process.

In [ ]:
def onehot(strings:Sequence[str], V, ctoi, max_len=None) -> torch.tensor:
    if max_len is None:
        max_len = maxlen(strings)
    X_onehot = torch.zeros(len(strings),len(V),max_len)
    for i,name in enumerate(strings):
        onehot = torch.zeros((len(V),max_len))
        for j,c in enumerate(name):
            onehot[ctoi[c],j] = 1
        X_onehot[i] = onehot
    return X_onehot

In [ ]:
sample = ['cat','a','at'] # always debug with a small representative example
o = onehot(sample, *vocab(sample))
o

In [ ]:
o[0,1].reshape(-1,1)

In [ ]:
X_train_onehot = onehot(X_train['name'], V, ctoi, max_len=max_len).to(device)
X_train_onehot[0].shape

In [ ]:
X_valid_onehot = onehot(X_valid['name'], V, ctoi, max_len=max_len).to(device)
X_valid_onehot[0].shape

## RNN model

Switching to W, U, V notation from $W_hh$ etc... from [Goodfellow and Yoshua Bengio and Aaron Courville book](https://www.deeplearningbook.org/contents/rnn.html)

We have a sequence of one-hot vectors for each word and need to predict a language for each sequence.  We need to know: vocab size (len of one hots), hidden len, and the number of target classes (langs).

We must combine a name's onehots into a single vector representing word then use a simple dense linear layer to make a prediction

$$
h^{(t)} = \text{ReLU}( W h^{(t-1)} + U x^{(t)} )
$$

where $t$ iterates through name length (or max pad length).

Note this is same as concatenating old state and current input vector and applying a single $W$ matrix of size nhidden x (nhidden+|V|):

$$
h^{(t)} = \text{ReLU}( W [h^{(t-1)};x^{(t)}] )
$$

The output is avail at every char but we only need the last one:

$$
y^{(t)} = V h^{(t)}
$$

This $V$ acts like the last dense linear layer which converts the hidden state to likelihood of each target class.

*What are the embeddings?* I think those are the final $h^{(t)}$ vectors, one of which is computed per name.  What are char-vec embeddings? Maybe $U$?

### Record-by-record (slow)

In [ ]:
class LastNameRNN_slow(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LastNameRNN_slow, self).__init__()
#         print("Model: ",input_size, hidden_size, output_size)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Help avoid vanishing gradient. Start with identity, which has
        # effect of summing char vector embeddings
        self.W  = torch.eye(hidden_size, hidden_size).double() #randn(hidden_size, hidden_size, std=0.01).double()
        self.U  = randn(hidden_size, input_size).double()
        self.V  = randn(output_size, hidden_size).double()
        self.W  = nn.Parameter(self.W)
        self.U  = nn.Parameter(self.U)
        self.V  = nn.Parameter(self.V)

    def forward(self, X):
#         print("X", X.shape)
        batch_size = X.shape[0]
        namelen = X.shape[2]
        # record softmax vec of output_size for each record
        o = torch.zeros((batch_size, self.output_size)).double().to(device)
        for i in range(batch_size):
            # Reset hidden state (history) at start of every record
            # Use same W and U matrices for all records until SGD update step
            h = torch.zeros((self.hidden_size, 1)).double().to(device)
            for j in range(namelen):  # for all chars in max name length
#                 print(h.shape, X[i].shape, X[i,:,j].shape, self.U.shape)
                h = self.W.mm(h) + self.U.mm(X[i,:,j].reshape(-1,1))
                h = torch.relu(h)  # better than sigmoid for vanishing gradient
            # we now have an h vector that is the embedding for the ith record
            # we have encoded/embedded the X[i] record into h
            # compute an output value, one per record
            ot = self.V.mm(h)
#             o[i] = F.softmax(ot.reshape(-1))
            o[i] = ot.reshape(-1)
        return o

In [ ]:
# test model
rnn = LastNameRNN_slow(input_size=len(V), hidden_size=10, output_size=len(y_cats)).to(device)
y_pred = rnn(torch.tensor(X_train_onehot[:100],device=device).double())
y_pred

## Train

In [ ]:
def ctrain(model:nn.Module, train_data:TensorDataset, valid_data:TensorDataset,
           epochs=350,
           test_size=0.20,
           learning_rate = 0.002,
           batch_size=32,
           weight_decay=1.e-4,
           loss_fn=F.cross_entropy,
           metric=accuracy_score,
           print_every=30):
    "Train a regressor"
    history = []
    train_loader = DataLoader(train_data, batch_size=batch_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#     optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    for ei in range(epochs): # epochs
        for bi, (batch_x, batch_y) in enumerate(train_loader): # mini-batch
#             if len(batch_x)!=batch_size:
#                 print(f"\tBatch {bi:3d} len {len(batch_x)}")
            y_prob = model(batch_x)
#             print("y pred", y_prob, "batch_y", batch_y)
            loss = loss_fn(y_prob, batch_y)

            optimizer.zero_grad()
            loss.backward() # autograd computes U.grad and M.grad
            optimizer.step()

        with torch.no_grad():
            o           = model(train_data.tensors[0])
            loss        = loss_fn(o, train_data.tensors[1])
            o           = model(valid_data.tensors[0])
            loss_valid  = loss_fn(o, valid_data.tensors[1])
            y_prob = model(train_data.tensors[0])
            y_prob = F.softmax(y_prob, dim=1)
            y_pred = torch.argmax(y_prob, dim=1)
            metric_train = metric(y_pred.cpu(), train_data.tensors[1].cpu())
            y_prob = model(valid_data.tensors[0])
            y_prob = F.softmax(y_prob, dim=1)
            y_pred = torch.argmax(y_prob, dim=1)
            metric_valid = metric(y_pred.cpu(), valid_data.tensors[1].cpu())

        history.append( (loss, loss_valid) )
        if ei % print_every == 0:
            print(f"Epoch {ei:3d} loss {loss:7.4f}, {loss_valid:7.4f}   accur {metric_train:4.3f}, {metric_valid:4.3f}")

    history = torch.tensor(history)
    return model, history

In [ ]:
rnn = LastNameRNN_slow(input_size=len(V),
                      hidden_size=100,
                      output_size=len(y_cats)).to(device)
subset=2000
train = TensorDataset(X_train_onehot[:subset].double().to(device), torch.tensor(y_train[:subset].values).long().to(device))
valid = TensorDataset(X_valid_onehot[:subset].double().to(device), torch.tensor(y_valid[:subset]).long().to(device))
model, history = ctrain(rnn, train, valid,
#                         loss_fn=torch.nn.BCELoss(),
                        loss_fn=F.cross_entropy,
                        metric=accuracy_score,
                        epochs=10,
                        learning_rate=.01,
                        weight_decay=0.00001,
                        batch_size=32,
                        print_every=1)

plot_history(history, yrange=(0,5))

### Timestep-by-step (fast)

In [ ]:
class LastNameRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LastNameRNN, self).__init__()
#         print("Model: ",input_size, hidden_size, output_size)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # combine W and U into W then cat h and input
        self.W  = nn.Linear(hidden_size+input_size, hidden_size)
        self.V  = nn.Linear(hidden_size, output_size)

    def forward(self, X):
#         print("X", X.shape)
        batch_size = X.shape[0]
        namelen = X.shape[2]
        # record softmax vec of output_size for each record
        o = torch.zeros((batch_size, self.output_size)).to(device)
        # now that we do all char j in a batch, h is a matrix
        h = torch.zeros((batch_size, self.hidden_size)).to(device)
        for j in range(namelen):  # for all chars in max name length
#                 print(h.shape, X[i].shape, X[i,:,j].shape, self.U.shape)
            xj = X[:,:,j] # jth char for all records in batch
#             print("W", self.W.weight.shape, "h", h.shape, "xj", xj.shape)
            combined = torch.cat((h, xj),dim=1)
#             print("combined", combined.shape)
            h = self.W(combined)
            h = torch.relu(h)  # better than sigmoid for vanishing gradient
        # we now have an h vector that is the embedding for the ith record
        # we have encoded/embedded the X[i] record into h
        # compute an output value, one per record
        ot = self.V(h)
#         print("ot shape", ot.shape)
#             o[i] = F.softmax(ot.reshape(-1))
#         o[i] = ot.reshape(-1)
        return ot

In [ ]:
rnn = LastNameRNN(input_size=len(V),
                  hidden_size=50,
                  output_size=len(y_cats)).to(device)
subset=5_000
train = TensorDataset(X_train_onehot[:subset].to(device), torch.tensor(y_train[:subset].values).long().to(device))
valid = TensorDataset(X_valid_onehot[:subset].to(device), torch.tensor(y_valid[:subset]).long().to(device))
model, history = ctrain(rnn, train, valid,
#                         loss_fn=torch.nn.BCELoss(),
                        loss_fn=F.cross_entropy,
                        metric=accuracy_score,
                        epochs=30,
                        learning_rate=.001,
                        weight_decay=0.000001,#002,
                        batch_size=64,
                        print_every=1)

plot_history(history, yrange=(0,2))

In [ ]:
class LastNameRNN_split(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LastNameRNN_split, self).__init__()
#         print("Model: ",input_size, hidden_size, output_size)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.W  = torch.eye(hidden_size, hidden_size)
        self.U  = randn(hidden_size, input_size)
        self.V  = randn(output_size, hidden_size)
        self.W  = nn.Parameter(self.W)
        self.U  = nn.Parameter(self.U)
        self.V  = nn.Parameter(self.V)

#         self.W  = nn.Linear(hidden_size+input_size, hidden_size)
#         self.V  = nn.Linear(hidden_size, output_size)

    def forward(self, X):
#         print("X", X.shape)
        batch_size = X.shape[0]
        namelen = X.shape[2]
        # record softmax vec of output_size for each record
        o = torch.zeros((batch_size, self.output_size)).to(device)
        # now that we do all char j in a batch, h is a matrix
        h = torch.zeros((self.hidden_size, batch_size)).to(device)
        for j in range(namelen):  # for all chars in max name length
            # xj is batchsize x |V| but U is hidden x |V| so need transpose
            xj = X[:,:,j].T # jth char dim for all records in batch
#             print(self.W.shape, h.shape, self.U.shape, xj.shape)
            h = self.W.mm(h) + self.U.mm(xj)
            h = torch.relu(h)  # better than sigmoid for vanishing gradient
        # we now have an h vector that is the embedding for the ith record
        # we have encoded/embedded the X[i] record into h
        # compute an output value, one per record
        ot = self.V.mm(h).T
#             o[i] = F.softmax(ot.reshape(-1))
#         print("ot shape", ot.shape, h.shape)
        return ot

In [ ]:
rnn = LastNameRNN_split(input_size=len(V),
                      hidden_size=50,
                      output_size=len(y_cats)).to(device)
subset=5_000
train = TensorDataset(X_train_onehot[:subset].to(device), torch.tensor(y_train[:subset].values).long().to(device))
valid = TensorDataset(X_valid_onehot[:subset].to(device), torch.tensor(y_valid[:subset]).long().to(device))
model, history = ctrain(rnn, train, valid,
#                         loss_fn=torch.nn.BCELoss(),
                        loss_fn=F.cross_entropy,
                        metric=accuracy_score,
                        epochs=30,
                        learning_rate=.001,
                        weight_decay=0.000001,#002,
                        batch_size=64,
                        print_every=1)

plot_history(history, yrange=(0,2))

Interesting.  Combined W and U seems to get better results, even though it should be equivalent. I must have some other different like precision or other numerical diffs. Oh duh. I'm using nn.Linear, which has a bias. wait it should be worse w/o bias. hmm..

## Error messages

For TensorDataset, if you see `TypeError: 'int' object is not callable`, it means you've passed a numpy array.

If it says "expected Long got Char", it might mean int8 not char.